In [1]:
%%capture
!pip3 install tensorflow==2.0.0-beta1

# Load the TensorBoard notebook extension
# Restart the jupyter notebook if you just installed TF 2.0 Beta
%reload_ext tensorboard

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import gym
import datetime
from tqdm import tqdm
import logging
import random

In [ ]:
!rm -rf ./logs/

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [ ]:
%%capture
!pip3 install -e gym-go

In [ ]:
go_env = gym.make('gym_go:go-v0', size='S', reward_method='heuristic')

In [ ]:
BOARD_SIZE = 7

# Machine Learning Models
Actor-Critic method

In [ ]:
from tensorflow.keras import layers

In [ ]:
def make_actor_critic(mode):
    assert mode == 'actor' or mode == 'critic'
    
    inputs = layers.Input(shape=(BOARD_SIZE, BOARD_SIZE, 4), name="board")
    valid_inputs = layers.Input(shape=(BOARD_SIZE**2 + 1,), name="valid_moves")
    
    # Dense block
    num_layers = 5
    growth_rate = 4
    x = inputs
    for i in range(num_layers):
        y = tf.keras.Sequential([
            layers.Conv2D(filters=4, kernel_size=5, padding="same", bias_initializer='ones'),
            layers.BatchNormalization(),
            layers.ReLU(),
        ], name='dense_layer_{}'.format(i))(x)
        x = layers.Concatenate()([x,y])
    
    board_distr = layers.Conv2D(filters=1, kernel_size=5, padding="same",
                                name="board_distr")(x)
            
    board_distr = layers.Flatten(name="board_distr_flattened")(board_distr)
    
    pass_value = layers.Conv2D(filters=1, kernel_size=BOARD_SIZE, padding="valid", 
                               bias_initializer=tf.keras.initializers.Constant(-4), 
                               name="pass_value")(x)
    
    pass_value = layers.Flatten(name="pass_value_flattened")(pass_value)
    
    moves = layers.Concatenate(name='all_moves')([board_distr, pass_value])
    
    if mode == 'actor':
        moves = layers.Softmax(name='moves_softmax')(moves)
    
    valid_moves = layers.Multiply(name='moves')([moves, valid_inputs])
    
    model = tf.keras.Model(inputs=[inputs, valid_inputs], outputs=valid_moves, name=mode)
    return model

In [ ]:
actor = make_actor_critic('actor')

In [ ]:
critic = make_actor_critic(mode='critic')

In [ ]:
opponent = tf.keras.models.clone_model(actor)
target_critic = tf.keras.models.clone_model(critic)

### Initialization of models 
should be random if the models are fresh

In [ ]:
def feed_forward_actor_critic(states, model):
    invalid_moves = states[:,:,:,2].reshape((-1,49))
    invalid_moves = np.insert(invalid_moves, BOARD_SIZE**2, 0, axis=1)
    valid_moves = 1 - invalid_moves
    moves = model([states.astype(np.float32), valid_moves.astype(np.float32)])
    return moves

In [ ]:
def state_responses(states):
    actor_moves = feed_forward_actor_critic(states, actor)
    critic_vals = feed_forward_actor_critic(states, critic)
    
    num_states = states.shape[0]
    
    fig = plt.figure()
    for i in range(num_states):
        plt.subplot(num_states,3,1 + 3*i)
        plt.axis('off')
        plt.title('Board')
        plt.imshow(states[i][:,:,[0,1,3]].astype(np.float))

        plt.subplot(num_states,3,2 + 3*i)
        plt.axis('off')
        plt.title('Actor\n{:.2f}L {:.2f}H {:.4f}P'
                  .format(np.min(actor_moves[i][:-1]), 
                          np.max(actor_moves[i][:-1]), 
                          actor_moves[i][-1]))
        plt.imshow(tf.reshape(actor_moves[i][:-1], (BOARD_SIZE, BOARD_SIZE)))

        plt.subplot(num_states,3,3 + 3*i)
        plt.axis('off')
        plt.title('Critic\n{:.1f}L {:.1f}H {:.1f}P'.
                  format(tf.math.reduce_min(actor_moves[i][:-1]), 
                         tf.math.reduce_max(critic_vals[i][:-1]), 
                         critic_vals[i][-1]))
        plt.imshow(tf.reshape(critic_vals[i][:-1],(BOARD_SIZE, BOARD_SIZE)))

    plt.tight_layout()
    return fig

In [ ]:
state = go_env.reset()
start_state = np.copy(state)
state, reward, done, info = go_env.step((3,5))
state, reward, done, info = go_env.step((5,3))

In [ ]:
states = state.transpose(1,2,0).reshape(1, BOARD_SIZE, BOARD_SIZE, 4)

In [ ]:
plt.show(state_responses(states))

# Training Setup

### Hyperparameters

In [ ]:
NUM_EPISODES = 1000
BATCH_SIZE = 128
MAX_STEPS = 3 * BOARD_SIZE**2
REPLAY_MEM_SIZE = 1024

ACTOR_LEARNING_RATE = 1e-4
OPPONENT_UPDATE = 100 

EPSILON = 1
EPSILON_DECAY = 0.995
EPSILON_MIN = 0.01

CRITIC_LEARNING_RATE = 1e-3
GAMMA = 0.75
TARGET_UPDATE = 1 # number of episodes to update the target critic model

In [ ]:
from collections import deque

In [ ]:
replay_mem = deque(maxlen=int(REPLAY_MEM_SIZE))

### Metrics and Tensorboard

Metrics

In [ ]:
actor_metrics = {}
for metric_key in ['loss']:
    actor_metrics[metric_key] = tf.keras.metrics.Mean('actor_{}'.format(metric_key), dtype=tf.float32)

critic_metrics = {}
for metric_key in ['loss']:
    critic_metrics[metric_key] = tf.keras.metrics.Mean('critic_{}'.format(metric_key), dtype=tf.float32)

Tensorboard

In [ ]:
summary_writers = {}
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
for summary_key in ['main']:
    log_dir = 'logs/actor_critic/{}/{}'.format(current_time, summary_key)
    summary_writers[summary_key] = tf.summary.create_file_writer(log_dir)

In [ ]:
import io
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

### Functions

In [ ]:
from sklearn import preprocessing
def random_weighted_action(move_weights):
    move_weights = preprocessing.normalize(move_weights, norm='l1')

    if np.sum(move_weights) <= 0:
        # Pass
        return None
    
    action_1d = np.random.choice(np.arange(BOARD_SIZE**2 + 1), p=move_weights[0])
    if action_1d == BOARD_SIZE**2:
        action = None
    else:
        action = (action_1d // BOARD_SIZE, action_1d % BOARD_SIZE)
    return action

In [ ]:
def get_action(policy, state, epsilon):
    """
    Gets an action based on exploration/exploitation
    """
    if state.shape[0] == 4:
        # State shape will be (BOARD_SIZE, BOARD_SIZE, 4)
        state = state.transpose(1,2,0)
            
    epsilon_choice = np.random.uniform()
    if epsilon_choice < EPSILON:
        # Random move
        logging.debug("Exploring a random move")
        invalid_moves = state[:,:,2].reshape((1,-1))
        invalid_moves = np.insert(invalid_moves, BOARD_SIZE**2, 0, axis=1)
        move_weights = 1 - invalid_moves
        
    else:
        # policy makes a move
        logging.debug("Exploiting policy's move")
        reshaped_state = state.reshape(1, BOARD_SIZE, BOARD_SIZE, 4).astype(np.float32)
        
        move_weights = feed_forward_actor_critic(reshaped_state, actor)
        move_weights = move_weights
        
    action = random_weighted_action(move_weights)
        
    return action

In [ ]:
def get_value_for_action(move_values, actions):
    '''
    Get value from board_values based on action, or take the passing_values if the action is None
    '''
    action_values = tf.gather_nd(move_values, [(i, a[0] * BOARD_SIZE + a[1]) if a is not None 
                                                    else (i, BOARD_SIZE**2) 
                                                    for i, a in enumerate(actions)])
    return action_values

In [ ]:
def get_batch_obs(size=BATCH_SIZE):
    '''
    Get a batch of orig_states, actions, states, rewards, terminals as np array out of replay memory
    '''
    batch = random.sample(replay_mem, size)
    batch = list(zip(*batch))
    orig_states = np.array(list(batch[0]), dtype=np.float32).transpose(0,2,3,1)
    actions = np.array(list(batch[1]))
    states = np.array(list(batch[2]), dtype=np.float32).transpose(0,2,3,1)
    rewards = np.array(list(batch[3]), dtype=np.float32)
    terminals = np.array(list(batch[4]), dtype=np.uint8)
    return orig_states, actions, states, rewards, terminals
    

In [ ]:
def max_action_vals(states, actval_func):
    '''
    The maximum action value (including passing) given the states
    '''
    move_vals = feed_forward_actor_critic(states, actval_func)
    max_vals = tf.reduce_max(move_vals, axis=1)
    return max_vals

In [ ]:
def calculate_state_vals(states, policy, actval_func):
    '''
    Get the expected value of states
    '''
    move_probs = feed_forward_actor_critic(states, policy)
    move_vals = feed_forward_actor_critic(states, actval_func)
    
    # state value is the expected critic value based on the actor's policy
    state_vals = tf.reduce_sum(move_probs * move_vals, axis=1)
    return state_vals

In [ ]:
mean_squared_error = tf.keras.losses.MeanSquaredError()

In [ ]:
critic_optimizer = tf.keras.optimizers.Adam(CRITIC_LEARNING_RATE)
def update_critic():
    """
    Optimizes the critic in one step and updates the critic loss metric
    """
    states, actions, next_states, rewards, terminals  = get_batch_obs()
    
    # get expected values for next state
    next_state_vals = calculate_state_vals(next_states, actor, target_critic)
    
    with tf.GradientTape() as tape:
        move_vals = feed_forward_actor_critic(states, critic)
        action_vals = get_value_for_action(move_vals, actions)
        val_loss = mean_squared_error(rewards + GAMMA * next_state_vals * (1-terminals), action_vals)
    
    critic_metrics['loss'](val_loss)
    
    # compute and apply gradients
    gradients = tape.gradient(val_loss, critic.trainable_variables)
    critic_optimizer.apply_gradients(zip(gradients, critic.trainable_variables))

In [ ]:
actor_optimizer = tf.keras.optimizers.Adam(ACTOR_LEARNING_RATE)
def update_actor():
    """
    Optimizes the actor in one step and updates the critic loss metric
    """
    states, actions, next_states, rewards, terminals = get_batch_obs()
    
    # State values (maximum action values)
    state_vals = max_action_vals(states, critic)
    
    # Next state values (maximum action values of next states)
    next_state_vals = max_action_vals(next_states, critic)
    
    # Advantage values
    avntg_vals = (rewards + GAMMA * next_state_vals * (1-terminals)) - state_vals
    
    with tf.GradientTape() as tape:
        move_probs = feed_forward_actor_critic(states, actor)
        action_probs = get_value_for_action(move_probs, actions)
        avntg_loss = - tf.math.log(action_probs + 1e-6) * avntg_vals
        avntg_loss = tf.math.reduce_mean(avntg_loss)
    
    # log the loss in tensorboard
    actor_metrics['loss'](avntg_loss)
        
    # compute and apply gradients
    gradients = tape.gradient(avntg_loss, actor.trainable_variables)
    actor_optimizer.apply_gradients(zip(gradients, actor.trainable_variables))

In [ ]:
def train(episode):
    """
    Plays out a game, and iteratively updates the models at each step
    """
    global EPSILON
    
    # Basic setup
    done = False
    num_of_turns = 0
    state = go_env.reset()
    rewards = []
    
    while not done and num_of_turns <= MAX_STEPS:
        # Copy state for memory
        orig_state = np.copy(state)
        
        black_action = get_action(actor, state, EPSILON)
        if black_action is None:
            logging.debug("Black (actor) passed")
            
        state, reward, done, info = go_env.step(black_action)
        num_of_turns += 1
        rewards.append(reward)
        
        # Update the critic and then actor if we have enough events
        if len(replay_mem) >= BATCH_SIZE:
            update_critic()        
            update_actor()
    
            # Update exploration/exploitation
            if EPSILON > EPSILON_MIN:
                EPSILON *= EPSILON_DECAY
                logging.debug("Epsilon decayed to {}".format(EPSILON))
            
        if done:
            # Add to memory
            replay_mem.append((orig_state, black_action, state, reward, done))
            break
            
        # opponent makes a move
        # swap the black and white layers
        temp = np.copy(state[0])
        state[0] = state[1]
        state[1] = temp
        # get action from opponent
        white_action = get_action(opponent, state, epsilon=0)
        if white_action is None:
            logging.debug("White (opponent) passed")

        state, reward, done, info = go_env.step(white_action)
        
        # Add to memory
        replay_mem.append((orig_state, black_action, state, reward, done))
        
        num_of_turns += 1
        rewards.append(reward)
    
    # Game ended
    return num_of_turns, rewards

# Train

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
for episode in tqdm(range(NUM_EPISODES)):
    # Reset all metrics
    for metric in actor_metrics.values():
        metric.reset_states()
    for metric in critic_metrics.values():
        metric.reset_states()
    
    if episode % OPPONENT_UPDATE == 0:
        opponent = tf.keras.models.clone_model(actor)
        logging.info("Updated opponent")
        
    if episode % TARGET_UPDATE == 0:
        target_critic = tf.keras.models.clone_model(critic)

    num_turns, rewards = train(episode)
    
    # Plot samples of states and actor, critic heatmaps
    num_samples = 1
    states, _, _, _,_ = get_batch_obs(num_samples)
    states = np.concatenate([states, start_state.transpose(1,2,0)
                             .reshape((-1,BOARD_SIZE, BOARD_SIZE, 4))], axis=0)
    fig = state_responses(states)
    
    # log results
    with summary_writers['main'].as_default():
        tf.summary.image("model heat maps", plot_to_image(fig), step=episode)
        
        tf.summary.scalar('last rewards', rewards[-1], step=episode)
        tf.summary.scalar('rewards sum', sum(rewards), step=episode)
        tf.summary.scalar('number of moves', num_turns, step=episode)
        tf.summary.scalar('critic loss', critic_metrics['loss'].result(), step=episode)
        tf.summary.scalar('actor loss', actor_metrics['loss'].result(), step=episode)
        tf.summary.scalar('epsilon', EPSILON, step=episode)
        

In [ ]:
actor.save_weights('tmp/actor.h5')
critic.save_weights('tmp/critic.h5')

# Evaluate

Test against a pretrained AI

In [ ]:
# TODO

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size='S')

state = go_env.reset()

done = False
while not done:
    go_env.render()
    
    # Actor's move
    action = get_action(actor, state, 0)
    
    state, reward, done, info = go_env.step(action)
    
    go_env.render()
    
    # Player's move
    player_moved = False
    while not player_moved:
        coords = input("Enter coordinates separated by space (`q` to quit)\n")
        if coords == 'q':
            done = True
            break
        coords = coords.split()
        try:
            row = int(coords[0])
            col = int(coords[1])
            print(row, col)
            state, reward, done, info = go_env.step((row, col))
            player_moved = True
        except Exception as e:
            print(e)